<a href="https://colab.research.google.com/github/Kyrylo-Bakumenko/Market-Simulation-Game/blob/main/MarketMan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
def generate_next_candle(open, actions, iterations, bias=0):
  base_freq = 1
  min_scale = 2
  max_scale = 5

  #assign price movements
  if iterations != 0:
    close_offset, low_offset, high_offset = np.random.normal(
        loc=bias,
        scale=min(max((iterations**np.random.randn())//(actions+10), min_scale), max_scale),
        size=3)
  else:
    close_offset, low_offset, high_offset = np.random.normal(loc=bias, scale=base_freq, size=3)

  #workout candlestick logistics
  close = max(open + close_offset, 0)
  low = open + low_offset
  high = open + high_offset
  if close >= open:
    high=max(high, close)
    low=min(low, open)
  else:
    high=max(high, open)
    low=min(low, close)
  return [close, low, high]

In [ ]:
def custom_rolling_mean_fill(df, column, source, window, price=100):
  for i in range(1,window):
    df.loc[i, column] = df[source][0:i].mean()
  df.loc[0, column] = price

In [ ]:
def custom_rolling_EMA_fill(df, column, source, window, price=100):
  for i in range(1,window):
    df.loc[i, column] = df[source][0:i].ewm(span=window).mean()
  df.loc[0, column] = price

In [ ]:
def close_positions(close_longs, open_longs, open_shorts, close_pos):
  # if close_longs: # close all long positions
  #   if close_pos[1] >= np.sum(open_longs[:, 1]): # no need to check volume, blanket close
  #     open_longs = np.zeros((1,2))
  #     open_longs[:, 0] = close_pos[0] - open_longs[:, 0]
  #     profit = np.sum(np.cumprod(open_longs, axis=1)[:, 1])
  #     return profit, open_longs, open_shorts
  # else: # close all short positions
  #   if close_pos[1] >= np.sum(open_longs[:, 1]): # no need to check volume, blanket close
  #     open_shorts = np.zeros((1,2))
  #     open_shorts[:, 0] -= close_pos[0]
  #     profit = np.sum(np.cumprod(open_shorts, axis=1)[:, 1])
  #     return profit, open_longs, open_shorts
  # close long positions starting from newest (greatest index) first
  profit = 0 # profit to be accumulated and returned
  if close_longs: # close all long positions
    while close_pos[1] > 0:
      if open_longs[-1, 1] >= close_pos[1]: # can be completed in this iteration
        profit += (open_longs[-1, 0]-close_pos[0])*close_pos[1]
        open_longs[-1, 1] -= close_pos[1]
        break # all volume in close_pos has been fulfilled
      else: # clear last open position, decrease volume
        profit += (open_longs[-1, 0]-close_pos[0])*open_longs[-1, 1] # increment profit
        close_pos[1] -= open_longs[-1, 1] # decrease closing position volume
        # remove closed position
        if open_longs.shape[0] > 1:
          open_longs = np.delete(open_longs, -1, 0) # delete closed position
        else: # set to [[0, 0]]
          open_longs = np.array([[0, 0]])
          open_shorts = np.append(open_shorts, [close_pos], axis=0)
          break # end loop, close_pos overflow assigned to opposite open positions
  else: # close all short positions
    while close_pos[1] > 0:
      if open_shorts[-1, 1] >= close_pos[1]: # can be completed in this iteration
        profit += (close_pos[0]-open_shorts[-1, 0])*close_pos[1]
        open_shorts[-1, 1] -= close_pos[1]
        break # all volume in close_pos has been fulfilled
      else: # clear last open position, decrease volume
        profit += (close_pos[0]-open_shorts[-1, 0])*open_shorts[-1, 1] # increment profit
        close_pos[1] -= open_shorts[-1, 1] # decrease closing position volume
        # remove closed position
        if open_shorts.shape[0] > 1:
          open_shorts = np.delete(open_shorts, -1, 0) # delete closed position
        else: # set to [[0, 0]]
          open_shorts = np.array([[0, 0]])
          open_longs = np.append(open_longs, [close_pos], axis=0)
          break # end loop, close_pos overflow assigned to opposite poen positions
      print(profit)
  return profit, open_longs, open_shorts

In [ ]:
open_longs = np.zeros((1, 2))
open_shorts = np.zeros((1, 2))
open_longs[0] = (125, 1)

open_longs = np.append(open_longs, [[100, 2]], axis=0)
open_longs = np.append(open_longs, [[110, 1.5]], axis=0)

print('Open Longs: \n', open_longs)
new_pos = np.array([125, 1])
print('\nClosing Position', new_pos, '\n')
prof, open_longs, open_shorts = close_positions(True, open_longs, open_shorts, new_pos)
print("Close_pos:\n", new_pos)
print("\nProfit: ", prof)
print("\nOpen Long Positions:\n", open_longs)
print("\nOpen Short Positions:\n", open_shorts)

Open Longs: 
 [[125.    1. ]
 [100.    2. ]
 [110.    1.5]]

Closing Position [125   1] 

Close_pos:
 [125   1]

Profit:  -15.0

Open Long Positions:
 [[125.    1. ]
 [100.    2. ]
 [110.    0.5]]

Open Short Positions:
 [[0. 0.]]


In [ ]:
price = 100 # starting price for stock-like
actions = 0 # number of actions selected by player
iterations = 0

scuffed = False # if true uses custom candlesticks instead of library

In [ ]:
# syntax practice for numpy stuff calculations
open_longs = np.zeros((1, 2))
open_longs[0] = (123, 1)

open_longs = np.append(open_longs, [[100, 2]], axis=0)
print(open_longs)
open_longs[:, 0] = 50 - open_longs[:, 0]
print(np.cumprod(open_longs, axis=1))
print(np.sum(np.cumprod(open_longs, axis=1)[:, 1]))
open_longs.shape

[[123.   1.]
 [100.   2.]]
[[ -73.  -73.]
 [ -50. -100.]]
-173.0


(2, 2)

In [ ]:
### Automated

# Game-ified Bias

bias = np.random.uniform(size = 1) # bias can be used for long term trend

# Simulation

Epochs = 100
Simulation_Step=5
View_Window=50

start_bal = 1_000
bal = start_bal
cumulative_profit = 0

price = 100
open = price
actions = 1
iterations = 0

SMA_length = 15
EMA1_length = 12
EMA2_length = 26
EMA_signal_line = 9

opens = [] # stores market opens
closes = [] # stores market closes
lows = [] # stores market lows
highs = [] # stores market highs
increased = [] # boolean; true if close>open for market day
iteration_count = [] # ??? I don't remember
sell_indices=[] # indicies at which sell actions were done
buy_indices=[] # indicies at which buy actions were done
share_volume = 0
margin_volume = 0
# legacy variables
# open_longs = np.zeros((1,2)) # stores open long positions
# open_shorts = np.zeros((1,2)) # stores open long positions
# #                   [[price1, volume1], [price2, volume2], ... ]

for i in range(Epochs):

  # logic computation
  ls = generate_next_candle(open, 1, iterations)
  if ls[0] <= 0:
    break;
  
  # update historical data
  opens.append(open)
  closes.append(ls[0])
  lows.append(ls[1])
  highs.append(ls[2])

  # print
  # print(ls)

  # count
  if scuffed:
    for i in range(4):
      iteration_count.append(iterations)
      increased.append(ls[0] >= open)
  else:
    iteration_count.append(iterations)
    increased.append(ls[0] >= open)
  iterations+=1
  
  # update next open to last close
  open = ls[0]


  ### oh yeah it's loop time ###
  o = np.asarray(opens)
  c = np.asarray(closes)
  l = np.asarray(lows)
  h = np.asarray(highs)
  g = np.asarray(increased)
  i = np.asarray(iteration_count)

  if scuffed:

    price_data = np.concatenate((o,c,l,h))
    d = {
      "Prices": price_data,
      "Epochs": i,
      "Green": g
    }
  else:
    d = {
        "Open":o,
        "Close":c,
        "Low":l,
        "High":h,
        "Epochs": i
    }


  df = pd.DataFrame(data=d)
  df.set_index("Epochs", inplace=True)

  # add volume
  df.insert(df.shape[1], "Volume", value=(abs((df["Close"]-df["Open"]))/np.random.uniform(low=0.7, high=1.43)*1_000_000))


  # add SMA - 15
  df.insert(df.shape[1], f'SMA{SMA_length}', value=df["Close"].rolling(SMA_length).mean())
  custom_rolling_mean_fill(df, f'SMA{SMA_length}', "Close", SMA_length)

  # work on MACD
  df.insert(df.shape[1], f'EMA{EMA1_length}', value=df["Close"].ewm(span=EMA1_length, adjust=False).mean())
  df.insert(df.shape[1], f'EMA{EMA2_length}', value=df["Close"].ewm(span=EMA2_length, adjust=False).mean())
  df.insert(df.shape[1], "MACD", df[f'EMA{EMA1_length}']-df[f'EMA{EMA2_length}'])
  df.insert(df.shape[1], f'EMA{EMA_signal_line}', value=df["MACD"].ewm(span=EMA_signal_line, adjust=False).mean())

  # do conditional coloring
  df.insert(df.shape[1], 'HistColors', value=df["MACD"]-df["EMA9"]>=0)
  for i in range(df.shape[0]):
    if df['HistColors'][i]:
      df.loc[i, 'HistColors'] = 'green'
    else:
      df.loc[i, 'HistColors'] = 'red'

  
  ### graph stuff

  # sub-plots for layered graphs
  fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
              vertical_spacing=0.1, subplot_titles=('DogeCoin', 'Volume', 'MACD & Histogram'), 
              row_width=[0.4, 0.4, 1])

  # cadnlestick chart of price
  fig.add_candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], 
                name="Price")
  
  # SMA line
  fig.add_trace(go.Scatter(x=df.index,
                            y=df[f'SMA{SMA_length}'],
                            mode="lines",
                            line=go.scatter.Line(color="turquoise"),
                            showlegend=True, name=f'SMA {SMA_length}'),
                row=1, 
                col=1)

  # Bar trace for volumes on 2nd row without legend
  fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend=False, marker={'color': 'sandybrown'}), row=2, col=1)

  # Volume by line chart -- OUTDATED
  # reference_line = go.Scatter(x=df.index,
  #                           y=df["Volume"],
  #                           mode="lines",
  #                           line=go.scatter.Line(color="gray"),
  #                           showlegend=True, name="Volume")

  # fig.add_trace(reference_line, row=1, col=1)

  # add MACD and Signal
  fig.add_trace(go.Scatter(x=df.index,
                            y=df["MACD"],
                            mode="lines",
                            line=go.scatter.Line(color="mediumslateblue"),
                            showlegend=True, name="MACD"),
                row=3, 
                col=1)
  
  
  fig.add_trace(go.Scatter(x=df.index,
                        y=df[f'EMA{EMA_signal_line}'],
                        mode="lines",
                        line=go.scatter.Line(color="orangered"),
                        showlegend=True, name="Signal Line"),
            row=3, 
            col=1)
  
  # add histogram
  fig.add_trace(go.Bar(x=df.index, y=df['MACD']-df["EMA9"], showlegend=False, name="Histogram", marker={'color': df['HistColors']}), row=3, col=1)

  # fig.update_layout(
  #   shapes = [dict(
  #       x0=20.5, x1=20.5, y0=0.56, y1=1, xref='x', yref='paper',
  #       line_width=2)]
  # )


    # annotations=[dict(
    #     x=50, y=0.56, xref='x', yref='paper',
    #     showarrow=False, xanchor='left', text='YOO! Look a bar!')]


  # get rid of slider
  fig.update_layout(xaxis_rangeslider_visible=False)
  fig.update_xaxes(
    range=[0-1, i+1],  # sets the range of xaxis
    constrain="domain",  # meanwhile compresses the xaxis by decreasing its "domain"
  )


  # add sell bars
  for sell_index in sell_indices:
    fig.add_trace(
    go.Scatter(
        x=[sell_index, sell_index],
        y=[df["High"].max(), df["Low"].min()],
        mode="lines",
        line=go.scatter.Line(color="red", width=2),
        showlegend=False),
        row=1,
        col=1
    )


  # add buy bars
  for buy_index in buy_indices:
    fig.add_trace(
      go.Scatter(
          x=[buy_index, buy_index],
          y=[df["High"].max(), df["Low"].min()],
          mode="lines",
          line=go.scatter.Line(color="green", width=2),
          showlegend=False),
          row=1,
          col=1
    )


  if i % Simulation_Step == 0:
    fig.show()
    # action input
    action = int(input("Buy (Long) or Sell (Short) or Nothing (1, -1, 0) "))
    actions += abs(action)
    if action is -1:
      sell_indices.append(i)
      volume = float(input(f'current DogeCoin price is: ${round(closes[i], 3)}\n Your bal: ${round(bal, 3)}\n How many shares to sell? ($)'))
      # total_position_value = np.sum(np.cumprod(open_longs, axis=1)[:, 1]))  ## not used right now, may be useful for portfolio toStrings later
      # update volumes
      if share_volume > 0:
        if share_volume >= volume:
          share_volume -= volume
        else:
          margin_volume += volume - share_volume
          share_volume = 0
      else:
        share_volume += volume
      # update bal
      bal += volume*closes[i]
          
    elif action is 1:
      buy_indices.append(i)
      volume = float(input(f'current DogeCoin price is: ${round(closes[i], 3)}\n Your bal: ${round(bal, 3)}\n How many shares to buy? ($)'))

      if margin_volume > 0:
        if margin_volume >= volume:
          margin_volume -= volume
        else:
          share_volume += volume - margin_volume
          margin_volume = 0
      else:
        margin_volume += volume
      # update bal
      bal -= volume*closes[i]
    
bal += share_volume*closes[-1]
bal -= margin_volume*closes[-1]
print(f'\n\n Your end of game return is ${round(bal-start_bal, 3)}',
      f'\nafter {actions} trades')



Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 1
current DogeCoin price is: $96.562
 Your bal: $1000
 How many shares to buy? ($)1


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) -1
current DogeCoin price is: $101.097
 Your bal: $903.4376480210275
 How many shares to sell? ($)1


...

Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 1
current DogeCoin price is: $117.751
 Your bal: $1124.509089899794
 How many shares to buy? ($)1


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


Buy (Long) or Sell (Short) or Nothing (1, -1, 0) 0


 Your end of game return is 6.757993522480774 
after 5 trades


In [ ]:
o = np.asarray(opens)
c = np.asarray(closes)
l = np.asarray(lows)
h = np.asarray(highs)
g = np.asarray(increased)
i = np.asarray(iteration_count)

if scuffed:

  price_data = np.concatenate((o,c,l,h))
  d = {
    "Prices": price_data,
    "Epochs": i,
    "Green": g
  }
else:
  d = {
      "Open":o,
      "Close":c,
      "Low":l,
      "High":h,
      "Epochs": i
  }


SMA_length = 15
EMA1_length = 12
EMA2_length = 26
EMA_signal_line = 9

df = pd.DataFrame(data=d)
df.set_index("Epochs", inplace=True)

# add volume
df.insert(df.shape[1], "Volume", value=(abs((df["Close"]-df["Open"]))/np.random.uniform(low=0.7, high=1.43)*1_000_000))


# add SMA - 15
df.insert(df.shape[1], f'SMA{SMA_length}', value=df["Close"].rolling(SMA_length).mean())
custom_rolling_mean_fill(df, f'SMA{SMA_length}', "Close", SMA_length)

# work on MACD
df.insert(df.shape[1], f'EMA{EMA1_length}', value=df["Close"].ewm(span=EMA1_length, adjust=False).mean())
df.insert(df.shape[1], f'EMA{EMA2_length}', value=df["Close"].ewm(span=EMA2_length, adjust=False).mean())
df.insert(df.shape[1], "MACD", df[f'EMA{EMA1_length}']-df[f'EMA{EMA2_length}'])
df.insert(df.shape[1], f'EMA{EMA_signal_line}', value=df["MACD"].ewm(span=EMA_signal_line, adjust=False).mean())

# do conditional coloring
df.insert(df.shape[1], 'HistColors', value=df["MACD"]-df["EMA9"]>=0)
for i in range(df.shape[0]):
  if df['HistColors'][i]:
    df.loc[i, 'HistColors'] = 'green'
  else:
    df.loc[i, 'HistColors'] = 'red'



In [ ]:


plt.figure()
if scuffed:
  sns.boxplot(x="Epochs", y="Prices", hue="Green", palette=["r", "g"], data=df);
  sns.lineplot(x=range(iterations), y=opens);
  plt.show()
else:
  # sub-plots for layered graphs
  fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
               vertical_spacing=0.1, subplot_titles=('DogeCoin', 'Volume', 'MACD & Histogram'), 
               row_width=[0.4, 0.4, 1])

  # cadnlestick chart of price
  fig.add_candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], 
                name="Price")
  
  # SMA line
  fig.add_trace(go.Scatter(x=df.index,
                            y=df[f'SMA{SMA_length}'],
                            mode="lines",
                            line=go.scatter.Line(color="turquoise"),
                            showlegend=True, name=f'SMA {SMA_length}'),
                row=1, 
                col=1)

  # Bar trace for volumes on 2nd row without legend
  fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend=False, marker={'color': 'sandybrown'}), row=2, col=1)

  # Volume by line chart -- OUTDATED
  # reference_line = go.Scatter(x=df.index,
  #                           y=df["Volume"],
  #                           mode="lines",
  #                           line=go.scatter.Line(color="gray"),
  #                           showlegend=True, name="Volume")

  # fig.add_trace(reference_line, row=1, col=1)

  # add MACD and Signal
  fig.add_trace(go.Scatter(x=df.index,
                            y=df["MACD"],
                            mode="lines",
                            line=go.scatter.Line(color="mediumslateblue"),
                            showlegend=True, name="MACD"),
                row=3, 
                col=1)
  
  
  fig.add_trace(go.Scatter(x=df.index,
                        y=df[f'EMA{EMA_signal_line}'],
                        mode="lines",
                        line=go.scatter.Line(color="orangered"),
                        showlegend=True, name="Signal Line"),
            row=3, 
            col=1)
  
  # add histogram
  fig.add_trace(go.Bar(x=df.index, y=df['MACD']-df["EMA9"], showlegend=False, name="Histogram", marker={'color': df['HistColors']}), row=3, col=1)

  # fig.update_layout(
  #   shapes = [dict(
  #       x0=20.5, x1=20.5, y0=0.56, y1=1, xref='x', yref='paper',
  #       line_width=2)]
  # )


    # annotations=[dict(
    #     x=50, y=0.56, xref='x', yref='paper',
    #     showarrow=False, xanchor='left', text='YOO! Look a bar!')]


  # get rid of slider
  fig.update_layout(xaxis_rangeslider_visible=False)


  # add demo buy & sell bars

  fig.add_trace(
    go.Scatter(
        x=[buy_index, buy_index],
        y=[df["High"].max(), df["Low"].min()],
        mode="lines",
        line=go.scatter.Line(color="green", width=2),
        showlegend=False),
        row=1,
        col=1
  )

  fig.add_trace(
  go.Scatter(
      x=[sell_index, sell_index],
      y=[df["High"].max(), df["Low"].min()],
      mode="lines",
      line=go.scatter.Line(color="red", width=2),
      showlegend=False),
      row=1,
      col=1
  )

  fig.show()

In [ ]:
      fig.add_trace(
        go.Scatter(
            x=[5, 5],
            y=[df["High"].max(), df["Low"].min()],
            mode="lines",
            line=go.scatter.Line(color="green", width=2),
            showlegend=False),
            row=1,
            col=1
      )
      fig.show()

In [ ]:
### adding text ###

In [ ]:
# import plotly.graph_objects as go
# import pandas as pd


# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

# fig = go.Figure(data=[go.Candlestick(x=df['Date'],
#                 open=df['AAPL.Open'], high=df['AAPL.High'],
#                 low=df['AAPL.Low'], close=df['AAPL.Close'])
#                       ])

# fig.update_layout(
#     title='The Great Recession',
#     yaxis_title='AAPL Stock',
#     shapes = [dict(
#         x0='2016-12-09', x1='2016-12-09', y0=0, y1=1, xref='x', yref='paper',
#         line_width=2)],
#     annotations=[dict(
#         x='2016-12-09', y=0.05, xref='x', yref='paper',
#         showarrow=False, xanchor='left', text='Increase Period Begins')]
# )

# fig.show()

In [ ]:
### User input

# price = 100
# actions = 1
# iterations = 0

# opens = []
# closes = []
# lows = []
# highs = []
# while True:
#   # open input
#   open = float(input("Open  "))
#   if open < 0:
#     break;

#   # action iput
#   action = float(input("Buy or Sell or Nothing (1, -1, 0) "))
#   if action is -1 or action is 1:
#     actions+=1

#   # logic computation
#   ls = generate_next_candle(open, actions, iterations)

#   # update historical data
#   opens.append(open)
#   closes.append(ls[0])
#   lows.append(ls[1])
#   highs.append(ls[2])

#   # print
#   print(ls)

#   # count
#   iterations+=1

In [ ]:
# plt.figure()
# sns.set_theme(style="darkgrid", palette="magma")
# sns.lineplot(x=range(iterations), y=opens);
# sns.lineplot(x=range(iterations), y=df[f'SMA{SMA_length}']);
# plt.legend(("Price", f'SMA{SMA_length}'));